Add train.txt, model.py 

In [1]:
!pip install pillow
!pip install torch
!pip install torchvision

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
!unzip /content/drive/MyDrive/dl_495/cross_age_dataset_cleaned_and_resized.zip >/dev/null
# !unzip /content/drive/MyDrive/dl_495/CACD2000.zip >/dev/null

In [1]:
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from PIL import Image
import torchvision
from torchvision import transforms
from model import Simple, Medium, AgeAlexNet

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f'Using {device}')

Using cuda


In [3]:
# Celeb dataset class 

class celeb_dataset(Dataset):
    def __init__(self, name_label_df, path_to_images, train=True):
        self.name_label_df = name_label_df
        self.path_to_images = path_to_images
        self.train = train
        
        transform = {"train": transforms.Compose([
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor()]),
            "valid": transforms.Compose([transforms.ToTensor()])}
        
        self.transform = transform

    def __len__(self):
        return len(self.name_label_df)

    def __getitem__(self, index):
        X = Image.open(self.path_to_images + self.name_label_df['file_name'].iloc[index])

        if self.train == True:
            X = self.transform["train"](X)
        else:
            X = self.transform["valid"](X)

        y = torch.tensor(self.name_label_df['class'].iloc[index]).long()

        return X, y

In [4]:
# Set up data
# df has file_names and associated classes 

image_path = 'cross_age_dataset_cleaned_and_resized/'
# image_path = 'CACD2000/'

df = pd.read_csv('train_edit.txt', sep=' ', header=None)
df.columns = ['file_name', 'class']

# # Changing class labels to make it a 3 class problem 
# classes_to_covert = list(df['class'])
# new = []

# for i in classes_to_covert:
#     if i == 0 or i == 1:
#         new.append(0)
#     elif i == 2 or i == 3:
#         new.append(1)
#     else:
#         new.append(2)

# df['3_class'] = np.array(new)
# df = df.drop(['class'], axis=1)
# df.columns = ['file_name', 'class']

# Train/validation split 
train = df.sample(frac=0.8,random_state=200)
valid = df.drop(train.index)

train = train.reset_index(drop=True)
valid = valid.reset_index(drop=True)

# Make Datasets and Dataloaders 
train_dataset = celeb_dataset(train, image_path, train=True)
val_dataset = celeb_dataset(valid, image_path, train=False)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=1)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=1)

In [5]:
train_dataset[3][0].shape

torch.Size([3, 224, 224])

In [6]:
# Model
# model = torchvision.models.vgg16_bn(pretrained=True)

# for param in model.parameters():
#     param.requires_grad = False

# model.classifier = nn.Sequential(
#         nn.Linear(25088 , 512),
#         nn.BatchNorm1d(512),
#         nn.Dropout(0.2),
#         nn.Linear(512 , 256),
#         nn.Linear(256 , 3))

# net = model
net = Simple()
net.to(device)

# Loss, optimizer 
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)

# Training loop 
overall_loss = []

for epoch in range(1):
    running_loss = 0.0
    for batch, (X,y) in enumerate(train_dataloader):
        X, y = X.to(device), y.to(device)
        optimizer.zero_grad()
        predictions = net(X)
        loss = criterion(predictions, y)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        if batch % 10 == 9:
            print(f'[{epoch + 1}, {batch + 1:5d}] loss: {running_loss}')
            overall_loss.append(loss.item())
            # torch.save(net.state_dict(), class_model_path_Alex + str(i) + '.pt')
            print('fin')

    print(f'[{epoch + 1}, {batch + 1:5d}] loss: {running_loss}')
    correct = 0
    total = 0

    with torch.no_grad():
        for X, y in val_dataloader:
            X, y = X.to(device), y.to(device)
            outputs = net(X)
            _, predicted = torch.max(outputs.data, 1)
            total += y.size(0)
            correct += (predicted == y).sum().item()
    print('validation_accuracy------------->' + str(correct / total))

    running_loss = 0.0

print('finished_training' + str(loss.item()))
print('mean_loss' + str(np.mean(overall_loss)))

[1,    10] loss: 23.07526060938835
fin
[1,    20] loss: 42.52188888192177
fin
[1,    30] loss: 57.891137927770615
fin
[1,    40] loss: 73.00183746218681
fin
[1,    50] loss: 89.65823730826378
fin
[1,    60] loss: 107.605323523283
fin
[1,    70] loss: 120.97991237044334
fin
[1,    80] loss: 138.75492987036705
fin
[1,    90] loss: 153.98434790968895
fin
[1,   100] loss: 171.74625226855278
fin
[1,   110] loss: 185.8916356265545
fin
[1,   120] loss: 202.010547965765
fin
[1,   130] loss: 218.1543935239315
fin
[1,   140] loss: 236.3541838824749
fin
[1,   150] loss: 252.44652912020683
fin
[1,   160] loss: 266.38225278258324
fin
[1,   170] loss: 281.0018511712551
fin
[1,   180] loss: 295.29258581995964
fin
[1,   190] loss: 309.68962582945824
fin
[1,   200] loss: 325.8988790214062
fin
[1,   210] loss: 342.7974699437618
fin
[1,   220] loss: 359.6921221911907
fin


FileNotFoundError: ignored